In [1]:
!pip install autogluon

In [2]:
import pandas as pd

filepath = "/home/jovyan/shared-public/ml_swe_monitoring_prod/snotel_ghcnd_stations_4yrs_all_cols_log10.csv"
chunksize = 1e5

columns = pd.read_csv(filepath, nrows=100).columns
chunked_data = pd.read_csv(filepath, chunksize=chunksize)
sample_data = pd.DataFrame(columns=columns)

for chunk in chunked_data:
    chunk.columns = columns
    chunk = chunk.sample(frac=0.01)
    sample_data = pd.concat([sample_data, chunk])

/tmp/ipykernel_1597/106610195.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sample_data = pd.concat([sample_data, chunk])


In [3]:
print(sample_data.shape)

(61556, 105)


In [4]:
chosen_columns = [
  "swe_value",
  'fsca',
  'air_temperature_tmmx', 
  'air_temperature_tmmn', 
  'potential_evapotranspiration', 
  'relative_humidity_rmax', 
  'Elevation',	
  'Slope',	
  'Curvature',	
  'Aspect',	
  'Eastness',	
  'Northness',
]

sample_data_subset = sample_data[chosen_columns]
sample_data_subset = sample_data_subset[sample_data_subset.swe_value != -999]

In [5]:
from autogluon.tabular import TabularDataset, TabularPredictor

tds = TabularDataset(sample_data_subset)

In [6]:
tds["swe_value"].describe()

count    41692.000000
mean         1.399192
std          5.354512
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         94.800000
Name: swe_value, dtype: float64

In [7]:
predictor = TabularPredictor(label="swe_value").fit(tds)

No path specified. Models will be saved in: "AutogluonModels/ag-20240822_213925"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Apr 23 20:32:19 UTC 2024
CPU Count:          4
Memory Avail:       23.95 GB / 30.89 GB (77.5%)
Disk Space Avail:   8589930436.68 GB / 8589934592.00 GB (100.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fa

[1000]	valid_set's rmse: 3.23229
[2000]	valid_set's rmse: 3.20495
[3000]	valid_set's rmse: 3.20643


	-3.1954	 = Validation score   (-root_mean_squared_error)
	6.49s	 = Training   runtime
	0.24s	 = Validation runtime
Fitting model: LightGBM ...
	-3.2376	 = Validation score   (-root_mean_squared_error)
	1.25s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-3.4427	 = Validation score   (-root_mean_squared_error)
	26.43s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-3.2912	 = Validation score   (-root_mean_squared_error)
	31.27s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-3.4526	 = Validation score   (-root_mean_squared_error)
	4.75s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-3.5811	 = Validation score   (-root_mean_squared_error)
	33.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-3.3665	 = Validation score   (-root_mean_squared_error)
	1.44s	 = Training   runtime
	0.02s	 = Validation runtime


In [8]:
results = predictor.leaderboard()

In [9]:
results

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-3.084965,root_mean_squared_error,0.314184,10.005421,0.000431,0.018088,2,True,12
1,LightGBMXT,-3.195382,root_mean_squared_error,0.241512,6.488254,0.241512,6.488254,1,True,3
2,LightGBM,-3.237612,root_mean_squared_error,0.020920,1.245535,0.020920,1.245535,1,True,4
3,LightGBMLarge,-3.258501,root_mean_squared_error,0.017612,2.182675,0.017612,2.182675,1,True,11
4,CatBoost,-3.291156,root_mean_squared_error,0.005328,31.266604,0.005328,31.266604,1,True,6
5,KNeighborsDist,-3.351344,root_mean_squared_error,0.033710,0.070869,0.033710,0.070869,1,True,2
6,XGBoost,-3.366450,root_mean_squared_error,0.016789,1.441106,0.016789,1.441106,1,True,9
7,RandomForestMSE,-3.442727,root_mean_squared_error,0.095697,26.426184,0.095697,26.426184,1,True,5
8,ExtraTreesMSE,-3.452625,root_mean_squared_error,0.084794,4.754848,0.084794,4.754848,1,True,7
9,NeuralNetTorch,-3.460290,root_mean_squared_error,0.012911,58.702260,0.012911,58.702260,1,True,10


In [10]:
!pip install pycaret

  Using cached scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.0
    Uninstalling scikit-learn-1.4.0:
      Successfully uninstalled scikit-learn-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-features 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-tabular 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.

In [13]:
from pycaret.regression import setup, compare_models
exp_reg = setup(sample_data_subset, target='swe_value', session_id=42) # target is the column we want to predict 

# Compare models
best_model = compare_models()

# Print the best model
print(best_model)

,Description,Value
0,Session id,42
1,Target,swe_value
2,Target type,Regression
3,Original data shape,"(41692, 12)"
4,Transformed data shape,"(41692, 12)"
5,Transformed train set shape,"(29184, 12)"
6,Transformed test set shape,"(12508, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.0150,12.0566,3.4621,0.5866,0.4167,1.9920,4.8630
lightgbm,Light Gradient Boosting Machine,0.9859,12.3339,3.5003,0.5775,0.4028,1.9964,38.9680
et,Extra Trees Regressor,1.0271,12.8852,3.5804,0.5574,0.4310,2.2354,1.4840
rf,Random Forest Regressor,1.0193,13.0136,3.5980,0.5528,0.4312,2.2223,5.9240
gbr,Gradient Boosting Regressor,1.0410,13.2892,3.6346,0.5441,0.4136,2.1103,2.0340
xgboost,Extreme Gradient Boosting,1.0722,13.4917,3.6659,0.5350,0.4393,2.1065,0.2120
lr,Linear Regression,1.1664,14.8513,3.8332,0.4953,0.4508,2.1656,0.4420
ridge,Ridge Regression,1.1664,14.8513,3.8332,0.4953,0.4508,2.1656,0.0180
en,Elastic Net,1.1595,14.8554,3.8336,0.4952,0.4468,2.1489,0.0190
br,Bayesian Ridge,1.1633,14.8515,3.8332,0.4952,0.4491,2.1649,0.0210
